## SEQ2Sql

Seq2sql implemented using BERT encoding instead of Glove.

### Import Libraries

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json
from transformers import BertForSequenceClassification, BertTokenizer, glue_convert_examples_to_features, BertModel
import numpy as np

### Import Preprocessed Dataset

Pre processed, tokenized WikiSql data is stored in './data/tokenized*'

In [5]:
dev_set_tokenized = pd.read_json('./data/tokenized_dev.jsonl',lines = True)
dev_set_tokenized.head()

In [6]:
dev_set = pd.read_json('./data/dev.jsonl',lines = True)
dev_set.head()

In [7]:
dev_tables = pd.read_json('./data/dev.tables.jsonl',lines = True)
dev_tables.head()

In [8]:
dev_tables_tokenized = pd.read_json('./data/tokenized_dev.tables.jsonl',lines = True)
dev_tables_tokenized.head()

Load the data into `sql_data` and `table_data`, the `table_data` needs to be a dictionary

In [9]:
sql_data = []
with open('./data/tokenized_dev.jsonl') as lines:
    for i,line in enumerate(lines):
        data = json.loads(line.strip())
        sql_data.append(data)

In [10]:
table_data = {}
with open('./data/tokenized_dev.tables.jsonl') as lines:
     for i, line in enumerate(lines):
        data =  json.loads(line.strip())
        table_data[data['id']] = data

In [11]:
print(len(table_data))

#### Universal function for loading the data

By passing the value `train` `dev` or `test` load the training dataset, validation dataset or the testing dataset respecitvily

In [5]:
def load_data_set(dataset_name):
    """
    Universal function to load the dataset
    Input: dataset_name - str {train, test, dev}
    Output: (sql_data, table_data) - (list, dictionary): containg the tokenized queries and the table dataset.
    """
    print(f"Loading {dataset_name} dataset")
    sql_file = './data/tokenized_'+dataset_name+'.jsonl'
    tables_file = './data/tokenized_'+dataset_name+'.tables.jsonl'
    sql_query_count = 0
    table_count = 0
    sql_data = []
    table_data = {}
    
    with open(sql_file) as lines:
        for line in lines:
            sql = json.loads(line.strip()) #remove white space before and after 
            sql_data.append(sql)
            sql_query_count += 1
    with open(tables_file) as lines:
        for line in lines:
            tab = json.loads(line.strip())
            table_data[tab['id']] = tab
            table_count += 1
    print(f"Loaded {sql_query_count} queries and {table_count} tables")
    return sql_data,table_data

In [6]:
train_sql, train_table = load_data_set('train')
print(f"{train_sql[0]}\n{train_table[train_sql[0]['table_id']]}")

Loading train dataset
Loaded 56355 queries and 18585 tables
{'phase': 1, 'table_id': '1-1000181-1', 'question': 'Tell me what the notes are for South Australia ', 'sql': {'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']], 'agg': 0}, 'tokenized_query': ['SELECT', 'notes', 'FROM', 'table_', 'WHERE', 'current', 'slogan', 'EQL', 'south', 'australia'], 'query': 'SELECT notes FROM table_ WHERE current slogan EQL south australia', 'tokenized_question': ['tell', 'me', 'what', 'the', 'notes', 'are', 'for', 'south', 'australia']}
{'id': '1-1000181-1', 'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'b

In [14]:
tuple(x for x in train_sql[0]['sql']['conds'])
#val_sql[0]['sql']['sel']

#### Function to create batch sequences.

Given a batch size this function creates batch sequences.  
The required indecies are passsed to idxes.

(This method of loading data is discouraged and need to implement `torch.utils.data.Dataset` and `Dataloader`)

In [15]:
def gen_batch_sequence(sql_data, table_data,idxes,start, end):
    # [3,5,7,8]
    question_seq = []
    column_seq = []
    number_of_col = []
    answer_seq = []
    query_seq = []
    ground_truth_cond_seq = []
    raw_data = []
    for i in range(start, end):
        sql = sql_data[idxes[i]]
        table_id = sql['table_id']
        question_seq.append(sql['tokenized_question'])
        column_seq.append(table_data[table_id]['tokenized_header'])
        number_of_col.append(len(table_data[table_id]['header']))
        answer_seq.append((sql['sql']['agg'],
                           sql['sql']['sel'],
                           len(sql['sql']['conds']),
                           tuple(x[0] for x in sql['sql']['conds']),
                            tuple(x[1] for x in sql['sql']['conds'])
                          ))
        query_seq.append(sql['tokenized_query'])
        ground_truth_cond_seq.append(sql['sql']['conds'])
        raw_data.append((sql['question'],table_data[table_id]['header'], sql['query']))
        
    return (question_seq, column_seq, number_of_col, answer_seq, query_seq, ground_truth_cond_seq,raw_data)

-----


#### Import BERT tokenizer 

In [16]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bertModel = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Freeze the bertModel
for params in bertModel.parameters():
    params.requires_grad = False

In [17]:
q_seq,col_seq, num_col, answer_seq, query_seq,gt_cond,rd = gen_batch_sequence(train_sql,train_table,[23,4,654],0,3)

In [18]:
col_seq[0]

In [19]:
for i, (one_q, one_col) in enumerate(zip(q_seq, col_seq)):
    #print(f"{i}: {one_q} || {one_col}")
    one_col_all = [x for toks in one_col for x in toks + ['[SEP]'] ]
    print(one_col_all)

In [20]:
text = ['[SEP]']
encode = tokenizer.encode_plus(text=text,add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
encode

In [21]:
print(q_seq)
batch_queries = [' '.join(x) for x in q_seq]
q_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs=batch_queries ,
                                #max_length = 60,
                                add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
bert_op = bertModel(**q_encode)
print(type(bert_op))
#print(q_encode["input_ids"])

In [22]:
print(bert_op.keys())

In [23]:
'''
Converting This: [['player'],
 ['no', '.'],
 ['nationality'],
 ['position'],
 ['years', 'in', 'toronto'],
 ['school/club', 'team']]
 
 Into : player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]
 
 for each element in the batch
'''

separator = '[SEP]'
header_batch_list=list(
                   map(lambda col: ' '.join([x for tok in col for x in tok + [separator]]), col_seq)
                    )
print(header_batch_list[0])

# for col in col_seq:
#     print(' '.join([x for tok in col for x in tok + [separator]]))
col_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs= header_batch_list,
                                 # max_length = 60,
                                add_special_tokens = True,
                                #truncation=True,
                                return_token_type_ids = True,
                                padding = 'longest', 
                                return_attention_mask = True,
                                return_tensors = 'pt')
col_encode['input_ids']

In [24]:
'''
Converting This: [['player'],
 ['no', '.'],
 ['nationality'],
 ['position'],
 ['years', 'in', 'toronto'],
 ['school/club', 'team']]
 
 Into : player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]
 
 for each element in the batch
 
 and combining the question with the above table header
 Eg:
 ['what', 'number', 'is', 'the', 'player', 'that', 'played', '1998-2001']
     and
 ['player', '[SEP]', 'no', '.', '[SEP]', 'nationality', '[SEP]', 'position', '[SEP]', 'years', 'in', 'toronto', '[SEP]', 'school/club', 'team', '[SEP]']
 
 into: what number is the player that played 1998-2001 [SEP] player [SEP] no . [SEP] nationality [SEP] position [SEP] years in toronto [SEP] school/club team [SEP]
'''

separator = '[SEP]'
header_batch_list=list(
                   map(lambda col: [x for tok in col for x in tok + [separator]], col_seq)
                    )
# print(header_batch_list[0])
# print(q_seq)

input_string = []
for i in range(len(header_batch_list)):
    input_string.append(' '.join(q_seq[i] +[separator]+ header_batch_list[i][:-1]))

print(input_string[0])    
tokenizer_args = { 'add_special_tokens' : True,
                   'return_token_type_ids' : True,
                   'padding' : 'longest', 
                   'return_attention_mask' : True,
                   'return_tensors' : 'pt'}
# for col in col_seq:
#     print(' '.join([x for tok in col for x in tok + [separator]]))
col_encode = tokenizer.batch_encode_plus(batch_text_or_text_pairs= input_string, **tokenizer_args)
col_encode['input_ids']

In [25]:
col_encode['input_ids'].shape

In [26]:
bert_op = bertModel(**col_encode)


In [27]:
print(bert_op.last_hidden_state)


In [28]:
print(list(col_encode.keys()))

In [29]:
x = torch.randn(2,3)
y = torch.randn(2,4)
z = torch.cat((x,y), dim=1)
print(f"X: {x}\nY: {y}\nZ: {z}")

In [30]:
con_cat_string = torch.cat((q_encode['input_ids'],col_encode['input_ids']),dim = 1)
print(con_cat_string)

In [31]:
tokenizer.convert_ids_to_tokens(col_encode['input_ids'][0])

In [32]:
## function to train the model
def epoch_train(model, optim, batch_size, sql_data, table_data):
    model.train()

## WordEmbedding Module

In [33]:
class WordEmbedding(nn.Module):
    def __init__(self, bert_encoder, max_length=None, separator = '[SEP]'):
        super(WordEmbedding, self).__init__()
        self.bert_tokenizer = BertTokenizer.from_pretrained(bert_encoder)
        self.bert_model = BertModel.from_pretrained(bert_encoder)
        self.max_length = max_length
        self.gpu = torch.cuda.is_available()
        self.seperator = separator
        
        self.bert_args = { 'add_special_tokens' : True,
                           'return_token_type_ids' : True,
                           'padding' : 'longest', 
                           'return_attention_mask' : True,
                           'return_tensors' : 'pt'}
        
        if(self.max_length != None):
            bert_args['max_length'] = self.max_length
            bert_args['padding'] = 'max_length'
        for param in self.bert_model.parameters():
            param.requires_grad = False
        assert list(self.bert_model.parameters())[0].requires_grad == False 
        
        if self.gpu:
            self.bert_model = self.bert_model.to('cuda')
    
    def gen_x_batch(self,q_batch,col_batch):
        '''
        Input: q_batch: list of tokenized query string i.e. List[List].
               col_batch: list of tokenzed header of the corresponding table header. List[List[List]]
        Output: ret_dict: a dictionary containing the 
        '''
        batch_queries = [' '.join(x) for x in q_batch]
        if batch_queries == []:
            print(q_batch, col_batch)
        '-------'
        header_batch_list=list(
                   map(lambda col: [x for tok in col for x in tok + [separator]], col_batch)
                    )

        input_string_list = []
        for i in range(len(header_batch_list)):
            input_string_list.append(' '.join(q_batch[i] +[separator]+ header_batch_list[i][:-1]))

        '-------'
        inp_encode = self.bert_tokenizer.batch_encode_plus(batch_text_or_text_pairs=input_string_list ,**self.bert_args)
        if self.gpu:
            for key in inp_encode.keys():
                inp_encode[key] = inp_encode[key].to('cuda')
        bert_op = self.bert_model(**inp_encode)
        
        # odict_keys(['last_hidden_state', 'pooler_output'])    
        return bert_op
        

In [35]:
#train_sql, train_table = load_data_set(data_set)
#def test_wordembed_module(data_set, batch_size = 32):
def test_wordembed_module(train_sql,train_table,batch_size = 32): 
    word_emb = WordEmbedding('bert-base-uncased')
    start = 0
    end = batch_size
    num_x = len(train_sql)
    idxes = np.random.permutation(num_x)
    i = 0
    while start < num_x:
        end = start+batch_size if start+batch_size <= num_x else num_x
        ret_tuple = gen_batch_sequence(train_sql,train_table,idxes, start, end)
        if ret_tuple[0] == []:
            print(start, end)
        bert_op = word_emb.gen_x_batch(ret_tuple[0],ret_tuple[1])
        last_hidden_state,pooler_output = bert_op.last_hidden_state, bert_op.pooler_output 
        
        assert last_hidden_state.size()[2] == 768
        assert last_hidden_state.size()[0] <= batch_size
        assert pooler_output.size()[1] == 768
        assert pooler_output.size()[0] <= batch_size
        
        if (i%10 == 0):
            print(f"\ni={i}\nSanity Check!!\n")
            print(f"Start: {start}\nEnd: {end}\n")
            print(f"Size of LastHidden Layer Size:{last_hidden_state.size()}\nPooler output size:{pooler_output.size()}\n\
            LastHidden Layer: {last_hidden_state}\nPooler output: {pooler_output}\n")
            
        i = i+1
        start = end
    
test_wordembed_module(train_sql,train_table)

In [1]:
from utils.utils import load_data_set
train_sql, train_table = load_data_set('train')

Loading train dataset
Loaded 56355 queries and 18585 tables


In [2]:
from model.wordEmbedding import test_wordembed_module

test_wordembed_module(train_sql, train_table)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



i=0
Sanity Check!!

Start: 0
End: 32

Size of LastHidden Layer Size:torch.Size([32, 60, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.7360, -0.3183, -0.1671,  ..., -0.0541,  0.3680,  0.6488],
         [ 0.1146,  0.1187, -0.3189,  ...,  0.4162,  0.3891,  0.2190],
         [-0.8834, -0.8451,  0.4053,  ...,  0.1687, -0.0132,  0.3803],
         ...,
         [ 0.2565, -0.5440,  0.3062,  ...,  0.2203, -0.1344, -0.6219],
         [-0.0052,  0.3801,  0.2257,  ...,  0.0346, -0.6065,  0.1337],
         [ 0.8852,  0.0332, -0.2031,  ...,  0.2007, -0.5840, -0.3072]],

        [[-0.5216,  0.2069, -0.3126,  ..., -0.2208, -0.1085,  0.4717],
         [-0.5596,  0.3364, -0.4079,  ...,  0.3152,  0.2703, -0.0337],
         [-0.4433, -0.4492, -0.2297,  ...,  0.1253, -0.3978,  0.7244],
         ...,
         [-0.5026,  0.2304,  0.1200,  ...,  0.0251, -0.4091,  0.3194],
         [-0.6771, -0.0647,  0.2924,  ...,  0.2989, -0.4582,  0.5327],
         [-0.5909,  0.2


i=30
Sanity Check!!

Start: 960
End: 992

Size of LastHidden Layer Size:torch.Size([32, 60, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.3537, -0.4447,  0.0796,  ..., -0.6434,  0.1948,  0.8515],
         [-0.9852, -0.1915, -0.1561,  ..., -0.1698,  0.9630, -0.2320],
         [ 0.8798,  0.1429,  0.6082,  ..., -1.0878,  0.5556,  0.3765],
         ...,
         [-0.1552, -0.3071,  0.3599,  ..., -0.3503,  0.1202,  0.3466],
         [-0.3092, -0.4811,  0.2938,  ..., -0.2282,  0.2688,  0.3767],
         [ 0.0138, -0.3922,  0.3843,  ..., -0.2030,  0.1247,  0.4370]],

        [[-0.4347, -0.6686,  0.0100,  ..., -0.0589,  0.4466,  0.8045],
         [-0.5453, -1.2426, -0.6027,  ...,  0.2512, -0.0025, -0.6876],
         [-0.6054, -1.0845,  0.7297,  ..., -0.3916, -0.0925,  0.3934],
         ...,
         [-0.2433, -0.2602,  0.3958,  ...,  0.0854,  0.1304,  0.2709],
         [-0.3735, -0.4544,  0.4562,  ...,  0.0743,  0.3404,  0.3983],
         [-0.3145, 


i=60
Sanity Check!!

Start: 1920
End: 1952

Size of LastHidden Layer Size:torch.Size([32, 56, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.4186, -0.0572,  0.0276,  ...,  0.1255,  0.0612,  0.7230],
         [ 0.3679,  0.5419, -0.6098,  ...,  0.1080,  0.7428,  0.3347],
         [-0.6119,  0.4308, -0.9248,  ..., -0.0860,  0.3031,  0.1657],
         ...,
         [-0.5177, -0.1298, -0.5078,  ...,  0.4387,  0.2018,  0.2584],
         [-0.2327,  0.0586, -0.2301,  ..., -0.0572,  0.0420,  0.4678],
         [-0.2754, -0.0618, -0.3754,  ..., -0.0844,  0.0877,  0.5161]],

        [[-0.5108,  0.0236,  0.1160,  ..., -0.3224,  0.1250,  0.7706],
         [-0.5650,  0.1232, -0.3174,  ...,  0.5099,  0.2544, -0.2093],
         [-0.4858, -0.2105, -0.1697,  ...,  0.1047, -0.4189,  0.6615],
         ...,
         [-0.5259, -0.2331,  0.4895,  ..., -0.1473,  0.0127,  0.2249],
         [-0.5382, -0.2782,  0.4199,  ..., -0.0694,  0.0462,  0.1749],
         [-0.5001


i=90
Sanity Check!!

Start: 2880
End: 2912

Size of LastHidden Layer Size:torch.Size([32, 76, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.8624, -0.0187, -0.1579,  ..., -0.4395,  0.2138,  0.7921],
         [-0.0390,  0.2538, -0.2520,  ...,  0.2850,  0.4865,  0.3346],
         [-0.4315, -0.8027, -0.0808,  ..., -0.2743,  0.1257,  0.3221],
         ...,
         [-0.9073, -0.1401,  0.1069,  ...,  0.1155,  0.4105,  0.3667],
         [-0.3261,  0.1077, -0.0609,  ...,  0.0315, -0.2177,  0.5651],
         [-0.8243, -0.1224,  0.0633,  ...,  0.0428,  0.3990,  0.4129]],

        [[-0.7829, -0.0038, -0.1921,  ..., -0.5213,  0.0922,  0.8662],
         [ 0.1274, -0.1997, -0.2169,  ...,  0.1693,  0.4594, -0.1525],
         [-0.1129, -0.3617, -0.4215,  ..., -0.2722, -0.8605,  0.5087],
         ...,
         [-0.3882,  0.1002,  0.2162,  ..., -0.2754, -0.2753,  0.3368],
         [-0.3701,  0.1535,  0.1294,  ..., -0.1883, -0.2319,  0.3708],
         [-0.9301

...


i=1630
Sanity Check!!

Start: 52160
End: 52192

Size of LastHidden Layer Size:torch.Size([32, 61, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-6.0269e-01,  7.1275e-04,  4.6279e-02,  ..., -4.5669e-01,
           2.3427e-01,  6.5363e-01],
         [-5.0927e-01, -5.3110e-02, -1.3220e-01,  ..., -7.7408e-02,
           1.9144e-01, -4.2693e-01],
         [-5.1627e-01, -4.8831e-01,  1.3056e-01,  ..., -2.0895e-01,
          -5.4963e-01,  5.5507e-01],
         ...,
         [-3.9472e-01, -2.4220e-01,  4.0400e-01,  ..., -4.0355e-02,
          -6.7121e-02,  1.5441e-01],
         [-1.7485e-01,  2.4590e-03,  4.0965e-01,  ..., -4.1821e-02,
           9.4490e-02, -2.0762e-01],
         [-1.0930e-01, -1.2356e-02,  3.8546e-01,  ..., -3.1004e-02,
           1.2122e-01, -1.9480e-01]],

        [[-3.2275e-01, -5.1727e-01,  1.1121e-01,  ..., -5.9681e-01,
           3.2106e-01,  7.8569e-01],
         [ 2.2852e-01, -6.1362e-02, -9.7381e-02,  ..., -3.6339e-02,
     


i=1660
Sanity Check!!

Start: 53120
End: 53152

Size of LastHidden Layer Size:torch.Size([32, 85, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-4.3408e-01,  7.4592e-02, -6.4419e-02,  ..., -4.8164e-01,
           2.6781e-01,  5.5573e-01],
         [-3.9077e-01,  9.2923e-03, -1.6465e-01,  ..., -1.0893e-01,
           2.8301e-01, -5.8216e-01],
         [-1.9799e-01, -4.6346e-01,  1.5239e-01,  ..., -2.9148e-01,
          -4.7083e-01, -1.5661e-01],
         ...,
         [-8.2186e-01,  2.4676e-02, -2.9425e-01,  ..., -1.1999e-01,
           2.6301e-01,  3.9343e-01],
         [-8.6009e-01, -1.5682e-01, -3.4591e-01,  ..., -8.6983e-03,
           3.3206e-01,  3.1158e-01],
         [-8.9876e-01, -4.5643e-02, -4.0072e-01,  ..., -1.2384e-01,
           2.7741e-01,  3.0819e-01]],

        [[-8.7794e-01, -4.0558e-02, -7.0775e-02,  ..., -2.3561e-01,
           6.7390e-02,  7.3793e-01],
         [-1.0536e+00,  3.9628e-01,  5.4010e-02,  ...,  1.1054e-01,
     


i=1680
Sanity Check!!

Start: 53760
End: 53792

Size of LastHidden Layer Size:torch.Size([32, 56, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.6800,  0.1472, -0.0612,  ..., -0.0862,  0.0815,  0.5751],
         [-0.1573,  0.2973, -0.5203,  ...,  0.2956,  0.3587,  0.1714],
         [-0.7481, -0.0746,  0.3875,  ...,  0.3198, -0.0905, -0.3032],
         ...,
         [-0.7036,  0.0768, -0.4195,  ...,  0.0624,  0.0120,  0.4629],
         [-0.6132, -0.0554,  0.1923,  ...,  0.2306, -0.2806,  0.2096],
         [-0.7201,  0.1226, -0.3066,  ..., -0.0124, -0.0835,  0.4601]],

        [[-0.2427, -0.2362, -0.0333,  ..., -0.3565,  0.2436,  0.5166],
         [ 0.1475,  0.0067, -0.2569,  ...,  0.0457,  0.3437,  0.1695],
         [-0.1664, -0.6126,  0.0745,  ...,  0.0389,  0.1448, -0.3017],
         ...,
         [-0.0808, -0.2543,  0.2518,  ..., -0.2044,  0.1870,  0.0999],
         [-0.1664, -0.4297, -0.1914,  ..., -0.0775,  0.2719,  0.1007],
         [-0.


i=1710
Sanity Check!!

Start: 54720
End: 54752

Size of LastHidden Layer Size:torch.Size([32, 66, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-0.5811, -0.0481, -0.0222,  ..., -0.4722,  0.0084,  0.8056],
         [-0.7959, -0.3710, -0.2735,  ...,  0.0794,  0.3657, -0.7167],
         [-0.5715, -0.2303, -0.0551,  ..., -0.2161, -0.2024,  0.8684],
         ...,
         [-0.4412, -0.4625,  0.1441,  ...,  0.0023,  0.2790,  0.1590],
         [ 0.2225, -0.3506,  0.2633,  ..., -0.1192,  0.2365,  0.2484],
         [-0.3227, -0.6951,  0.0816,  ...,  0.0335,  0.3247, -0.0191]],

        [[-0.8617,  0.0148, -0.1581,  ..., -0.3013,  0.1721,  0.6167],
         [-0.5601,  0.0648, -0.2381,  ..., -0.0861, -0.0984, -0.5380],
         [-1.0396, -0.6927, -0.4200,  ...,  0.4761, -0.9501,  0.3220],
         ...,
         [-0.8117, -0.2397,  0.1281,  ..., -0.1736,  0.2829,  0.4800],
         [-0.7296, -0.1101, -0.0423,  ..., -0.0731,  0.2793,  0.4489],
         [-0.


i=1740
Sanity Check!!

Start: 55680
End: 55712

Size of LastHidden Layer Size:torch.Size([32, 50, 768])
Pooler output size:torch.Size([32, 768])
            LastHidden Layer: tensor([[[-1.0093, -0.1271, -0.2306,  ..., -0.5149,  0.1722,  1.0066],
         [-0.5753,  0.0229, -0.3885,  ...,  0.1471,  0.0945, -0.2974],
         [-0.4477, -0.3775, -0.6069,  ..., -0.3100, -0.5189,  0.6909],
         ...,
         [-1.0947, -0.1750,  0.0806,  ..., -0.2064,  0.2015,  0.8802],
         [-1.0987, -0.2349,  0.0762,  ..., -0.2285,  0.2484,  0.8604],
         [-0.9940, -0.1528,  0.0529,  ..., -0.3231,  0.0141,  0.7753]],

        [[-0.0446, -0.0382,  0.2009,  ..., -0.1000,  0.6883,  0.4686],
         [ 0.1332, -0.4041,  0.0233,  ...,  0.3974,  0.4271, -0.1816],
         [-0.0781, -0.6564,  0.8106,  ..., -0.3620, -0.2321,  0.0646],
         ...,
         [-0.1395, -0.1115,  0.1745,  ...,  0.4431,  0.4560, -0.4498],
         [-0.1269, -0.0154,  0.4235,  ...,  0.3803,  0.4507, -0.0520],
         [-0.